In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import aiohttp
import asyncio
import nest_asyncio


In [2]:
url = 'https://www.gourmettraveller.com.au/dining-out/restaurant-guide/the-top-82-restaurants-in-australia-for-2023-20320'
text = requests.get(url).text
soup = BeautifulSoup(text, 'html.parser')


In [3]:
restaurant = pd.DataFrame(columns=['Name', 'City', 'url', 'address', 'phone', 'web', 'chefs', 'price_guide'])
for i, line in enumerate(soup.body.find_all('a', class_="")):
    try:
        restaurant.loc[i]={ 'Name': line.get_text().split(",")[0], 'City': line.get_text().split(",")[1],'url': (line.get('href'))}
    except IndexError:
        pass
restaurant




,Name,City,url,address,phone,web,chefs,price_guide
0,Margaret,Sydney,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
1,Beach,Byron Bay,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
2,Bentley,Sydney,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
3,Berowra Waters Inn,Berowra,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
4,Cafe Paci,Sydney,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
77,O.My,Beaconsfield,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
78,Omnia,South Yarra,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
79,Provenance,Beechworth,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN
80,Tulum,Balaclava,https://www.gourmettraveller.com.au/dining-out...,NaN,NaN,NaN,NaN,NaN


In [5]:
url = 'https://www.gourmettraveller.com.au/dining-out/restaurant-guide/margaret-sydney-review-20181'

# download the HTML content of the page
response = requests.get(url)
html = response.text

# parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# find the element containing the address
soup.find('h2', text='ABOUT').findNext('div', class_="content-body").get_text('\n').split('\n')


['Margaret',
 'Cnr Bay St & Guilfoyle Ave, Double Bay',
 '(02) 9068 8888',
 'margaretdoublebay.com',
 'Chefs',
 ' Neil Perry & Richard Purdue',
 'Price guide',
 ' $$$',
 'Bookings',
 ' Recommended',
 'Wheelchair access',
 ' Yes',
 "This review was made independently for the Gourmet Traveller Restaurant Guide. The guide's reviewers visit unannounced and pay their way. To learn more about the process, ",
 'click here',
 '.']

In [63]:
inf_list =[]
async def fetch(url, session):
    async with session.get(url) as response:
        return await response.text()

async def parse(url, session):
    # download the HTML content of the page
    html = await fetch(url, session)

    # parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # find the element containing the address
    address_element = soup.find('h2', text='ABOUT').findNext('div', class_="content-body")
    address = address_element.get_text('\n').split('\n')

    phone_number_pattern = re.compile(r'(\(\d{2}\) \d{4} \d{4}|\d{4} \d{3} \d{3})')


    if phone_number_pattern.search(address[2])==None:
        address = address[:2] + [''] + address[2:]



    print(f'count: {len(address)}')
    print(f'Address: {address}')
    print()
    return address

async def main(restaurant_urls):
    async with aiohttp.ClientSession() as session:
        tasks = [parse(url, session) for url in restaurant_urls]
        inf_list.append(await asyncio.gather(*tasks))

        
nest_asyncio.apply()
restaurant_urls = restaurant['url'].to_list()
asyncio.run(main(restaurant_urls))

count: 15
Address: ['Cafe Paci', '131 King St, Newtown', '(02) 9550 6196', 'cafepaci.com.au', 'Chef', ' Pasi Petänen', 'Price guide', ' $$', 'Bookings', ' Recommended', 'Wheelchair access', ' Yes', "This review was made independently for the Gourmet Traveller Restaurant Guide. The guide's reviewers visit unannounced and pay their way. To learn more about the process, ", 'click here', '.']

count: 15
Address: ['Porcine', '268 Oxford St, Paddington', '0423 015 032', 'Porcine.com.au', 'Chef', ' Nik Hill', 'Price guide', ' $$', 'Bookings', ' Recommended', 'Wheelchair access', ' No', "This review was made independently for the Gourmet Traveller Restaurant Guide. The guide's reviewers visit unannounced and pay their way. To learn more about the process, ", 'click here', '.']

count: 15
Address: ['Aru', '268 Little Collins St, Melbourne', '(03) 9939 8113', 'aru.net.au', 'Chef', ' Khanh Nguyen', 'Price guide', ' $$$', 'Bookings', ' Recommended', 'Wheelchair access', ' Yes', "This review was ma

In [68]:
for i, line in enumerate (inf_list[0]):
    restaurant['address'][i] = line[1]
    restaurant['phone'][i] = line[2]
    restaurant['web'][i] = line[3]
    restaurant['chefs'][i] = line[5]
    restaurant['price_guide'][i] = line[7]

C:\Users\Dell\AppData\Local\Temp\ipykernel_3156\1295247738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant['address'][i] = line[1]
C:\Users\Dell\AppData\Local\Temp\ipykernel_3156\1295247738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant['phone'][i] = line[2]
C:\Users\Dell\AppData\Local\Temp\ipykernel_3156\1295247738.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant['web'][i] = line[3]


In [74]:
restaurant.drop('url', axis=1)

,Name,City,address,phone,web,chefs,price_guide
0,Margaret,Sydney,"Cnr Bay St & Guilfoyle Ave, Double Bay",(02) 9068 8888,margaretdoublebay.com,Neil Perry & Richard Purdue,$$$
1,Beach,Byron Bay,"2 Massinger St, Byron Bay",1300 583 766,beachbyronbay.com.au,Alanna Sapwell,$$
2,Bentley,Sydney,"27 O'Connell St, Sydney",(02) 8214 0505,thebentley.com.au,Brent Savage & Aiden Stevens,$$$$
3,Berowra Waters Inn,Berowra,"1 Calabash Pt, Berowra Waters",(02) 9456 1027,berowrawatersinn.com,Brian Geraghty & Lauren Eldridge,$$$$
4,Cafe Paci,Sydney,"131 King St, Newtown",(02) 9550 6196,cafepaci.com.au,Pasi Petänen,$$
...,...,...,...,...,...,...,...
77,O.My,Beaconsfield,"70 Princes Hwy, Beaconsfield",(03) 9769 9000,omyrestaurant.com.au,Blayne Bertoncello,$$$$
78,Omnia,South Yarra,"625 Chapel St, South Yarra",(03) 8080 8080,omniabistro.com.au,Stephen Nairn,$$$
79,Provenance,Beechworth,"86 Ford St, Beechworth",(03) 5728 1786,theprovenance.com.au,Michael Ryan,$$$$
80,Tulum,Balaclava,"217 Carlisle St, Balaclava",(03) 9525 9127,tulumrestaurant.com.au,Coskun Uysal,$$


In [75]:
import plotly.express as px

In [87]:
states = pd.DataFrame(restaurant['City'].value_counts()).reset_index()
states.columns = ['state_name', 'count']